In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import date
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
data = {
    'plots': {},
    'stats': {}
}

In [ ]:
confirmed = pd.read_csv('datasets/confirmed_transposed.csv', index_col=0)
deaths = pd.read_csv('datasets/deaths_transposed.csv', index_col=0)
recovered = pd.read_csv('datasets/recovered_transposed.csv', index_col=0)

In [ ]:
def sort_columns_on_last_row(df):
    sorted_indicies = (-df.loc[df.last_valid_index()]).argsort()
    return df[df.columns[sorted_indicies]]

In [ ]:
def over_threshold(df, threshold):
    mask = (df.iloc[-1] > threshold).index[df.iloc[-1] > threshold]
    over_threshold = df[mask]
    return sort_columns_on_last_row(over_threshold)

In [ ]:
threshold = 500
confirmed_over_threshold = over_threshold(confirmed, threshold)
fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Countries with over {threshold} confirmed cases due to COVID-19'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            showgrid=True,
            automargin=True,
            gridwidth=1,
            gridcolor='rgb(200,200,200)'
        ),
        xaxis=go.layout.XAxis(
            showgrid=True,
            automargin=True,
            gridwidth=1,
            gridcolor='rgb(200,200,200)',
            dtick=4
        ),
        margin={
            'l': 0, 'r': 0, 'pad': 0
        }
    )
)

for i, column in enumerate(confirmed_over_threshold):
    color, show = get_color(column, i, 8)
    fig.add_trace(go.Scatter(
        x=confirmed_over_threshold[column].index, 
        y=confirmed_over_threshold[column].values,
        name=column,
        marker_color=color,
        line=dict(width=1, dash='solid' if show else 'dot')
    ))
fig.show()
data['plots']['figure_1'] = fig.to_json()

In [ ]:
def get_from_first_occurrence(df, country, above_value):
    above = df[country][df[country] > above_value]
    return np.arange(0, len(above)), above.values

In [ ]:
threshold = 5
confirmed_over_threshold = over_threshold(deaths, threshold)
fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Countries with over {threshold} confirmed cases due to COVID-19'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            showgrid=True,
            automargin=True,
            gridwidth=1,
            gridcolor='rgb(200,200,200)'
        ),
        xaxis=go.layout.XAxis(
            showgrid=True,
            automargin=True,
            gridwidth=1,
            gridcolor='rgb(200,200,200)',
            dtick=4
        ),
        margin={
            'l': 0, 'r': 0, 'pad': 0
        }
    )
)

for i, column in enumerate(confirmed_over_threshold):
    color, show = get_color(column, i, 8)
    fig.add_trace(go.Scatter(
        x=confirmed_over_threshold[column].index, 
        y=confirmed_over_threshold[column].values,
        name=column,
        marker_color=color,
        line=dict(width=1, dash='solid' if show else 'dot')
    ))
fig.show()
data['plots']['figure_2'] = fig.to_json()

In [ ]:
threshold = 5
deaths_over_threshold = over_threshold(deaths, threshold)
deaths_china = deaths_over_threshold['China']
deaths_over_threshold = deaths_over_threshold.drop(['China'], axis=1)

fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Top 6 countries with over {threshold} deaths, aligned with second registered death'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            type='log', showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)'
        ),
        xaxis=go.layout.XAxis(showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)', dtick=4),
        margin={
            'l': 0, 'r': 0, 'pad': 0
        }
    )
)


fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction='left',
            buttons=[
                dict(
                    label="Logarithmic",
                    method="relayout",
                    args=["yaxis.type", 'log']
                ),
                dict(
                    label="Linear",
                    method="relayout",
                    args=["yaxis.type", 'lin']
                ),
            ],
            pad={"r": 0, "t": 0},
            showactive=True,
            x=0,
            xanchor="left",
            y=-0.1,
            yanchor="top"
        )
    ]
)

max_count = 0
for i, column in enumerate(deaths_over_threshold):
    x, y = get_from_first_occurrence(deaths_over_threshold, column, 1)
    if len(x) > max_count:
        max_count = len(x)
    color, show = get_color(column, i, 6)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=column,
        mode='lines',
        marker_color=color,
        line=dict(width=1, dash='solid' if show else 'dot'),
        visible=True if show else 'legendonly'
    ))

color, show = get_color('China')
fig.add_trace(go.Scatter(
    x=np.arange(0, max_count), 
    y=np.array([2, 2, 4, 7, 11] + deaths_china.values.tolist())[:max_count],
    name='China',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))
fig.show()
data['plots']['figure_3'] = fig.to_json()

In [ ]:
threshold = 5
deaths_over_threshold = over_threshold(deaths, threshold)
deaths_china = deaths_over_threshold['China']
deaths_over_threshold = deaths_over_threshold.drop(['China'], axis=1)

fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Countries with over {threshold} deaths, aligned with second registered death'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            type='log', showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)'
        ),
        xaxis=go.layout.XAxis(showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)', dtick=4),
        margin={
            'l': 0, 'r': 0, 'pad': 0
        }
    )
)


fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction='left',
            buttons=[
                dict(
                    label="Logarithmic",
                    method="relayout",
                    args=["yaxis.type", 'log']
                ),
                dict(
                    label="Linear",
                    method="relayout",
                    args=["yaxis.type", 'lin']
                ),
            ],
            pad={"r": 0, "t": 0},
            showactive=True,
            x=0,
            xanchor="left",
            y=-0.1,
            yanchor="top"
        )
    ]
)

color, show = get_color('China')
fig.add_trace(go.Scatter(
    x=np.arange(0, len(deaths_china)), 
    y=np.array([2, 2, 4, 7, 11] + deaths_china.values.tolist()),
    name='China',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))

for i, column in enumerate(deaths_over_threshold):
    x, y = get_from_first_occurrence(deaths_over_threshold, column, 1)
    color, show = get_color(column, i, 8)
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        name=column,
        mode='lines',
        marker_color=color,
        line=dict(width=1, dash='solid' if show else 'dot')
    ))
fig.show()
data['plots']['figure_4'] = fig.to_json()

In [ ]:
hubei_lockdown = '2020-01-23'
lombardy_lockdown = '2020-03-08'
spain_lockdown = '2020-03-14'
uk_lockdown = '2020-03-20'
norway_lockdown = '2020-03-12'
france_lockdown = '2020-03-17'

In [ ]:
def align_series(series, lockdown, latest_lockdown):
    delta = np.timedelta64(pd.to_datetime(latest_lockdown) - pd.to_datetime(lockdown), 'D')
    x = np.arange(0, delta + len(series)).astype(int)
    y = np.append(np.full((delta / np.timedelta64(1, 'D')).astype(int), None), series.values)
    return x, y

In [ ]:
np.where(pd.to_datetime(deaths.index).values == np.datetime64(pd.to_datetime(spain_lockdown)))

In [ ]:
np.where(pd.to_datetime(deaths.index).values == np.datetime64(pd.to_datetime(hubei_lockdown)))

In [ ]:
lockdown_index

In [ ]:
latest_lockdown = uk_lockdown
lockdown_index = np.where(pd.to_datetime(deaths.index).values == np.datetime64(pd.to_datetime(latest_lockdown)))[0][0]
cutoff = 40
current_max = max(deaths.iloc[-1])

fig = go.Figure(
    layout=go.Layout(
        title=go.layout.Title(text=f'Number of deaths due to COVID-19 when countries initiated lockdown'),
        plot_bgcolor='rgb(255,255,255)',
        yaxis=go.layout.YAxis(
            showgrid=True, gridwidth=1, gridcolor='rgb(200,200,200)', range=[0, 500]
        ),
        xaxis=go.layout.XAxis(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='rgb(200,200,200)', 
            dtick=4,
            range = [cutoff, 60]
        ),
        margin={
            'l': 0, 'r': 0, 'pad': 0
        }
    )
)

x, y = align_series(deaths['China'], hubei_lockdown, spain_lockdown)

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction='left',
            buttons=[
                dict(
                    label="Zoom in",
                    method="relayout",
                    args=[{"xaxis.range": [cutoff, 60], "yaxis.range": [0, 550]}]
                ),
                dict(
                    label="Zoom out",
                    method="relayout",
                    args=[{"xaxis.range": [cutoff, len(x)], "yaxis.range": [0, current_max]}]
                ),
            ],
            pad={"r": 0, "t": 0},
            showactive=True,
            x=0,
            xanchor="left",
            y=-0.1,
            yanchor="top"
        )
    ]
)


x, y = align_series(deaths['China'], hubei_lockdown, latest_lockdown)
color, show = get_color('China')
fig.add_trace(go.Scatter(
    x=x[cutoff:], 
    y=y[cutoff:],
    name='China',
    mode='lines',
    marker_color='rgb(255,0,150)',
    line=dict(width=1)
))

x, y = align_series(deaths['Italy'], lombardy_lockdown,latest_lockdown)
color, show = get_color('Italy')
fig.add_trace(go.Scatter(
    x=x[cutoff:], 
    y=y[cutoff:],
    name='Italy',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))

x, y = align_series(deaths['Spain'], spain_lockdown, latest_lockdown)
color, show = get_color('Spain')
fig.add_trace(go.Scatter(
    x=x[cutoff:], 
    y=y[cutoff:],
    name='Spain',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))

x, y = align_series(deaths['United Kingdom'], uk_lockdown, latest_lockdown)
color, show = get_color('United Kingdom')
fig.add_trace(go.Scatter(
    x=x[cutoff:], 
    y=y[cutoff:],
    name='United Kingdom',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))

x, y = align_series(deaths['France'], france_lockdown, latest_lockdown)
color, show = get_color('France')
fig.add_trace(go.Scatter(
    x=x[cutoff:], 
    y=y[cutoff:],
    name='France',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))

x, y = align_series(deaths['Norway'], norway_lockdown, latest_lockdown)
color, show = get_color('Norway')
fig.add_trace(go.Scatter(
    x=x[cutoff:], 
    y=y[cutoff:],
    name='Norway',
    mode='lines',
    marker_color=color,
    line=dict(width=1)
))

fig.add_trace(go.Scatter(
    x=[lockdown_index, lockdown_index],
    y=[0, current_max],
    name='Time of lockdown',
    mode="lines",
    marker_color="rgb(100,100,100)",
    line=dict(dash="dot")
))

fig.show()
data['plots']['figure_5'] = fig.to_json()

In [ ]:
def align_on_date(series, future, pivot):
    return series.loc[pd.to_datetime(series.index) >= pd.to_datetime(future) - np.timedelta64(pd.to_datetime(future) - pd.to_datetime(pivot), 'D')]

In [ ]:
np.timedelta64(pd.to_datetime(spain_lockdown) - pd.to_datetime(deaths_china.index[0]), 'D') - np.timedelta64(pd.to_datetime(lombardy_lockdown) - pd.to_datetime(deaths_china.index[0]), 'D')

In [ ]:
def shifted_index(series, lockdown, latest_lockdown):
    reference_delta = np.timedelta64(pd.to_datetime(latest_lockdown) - pd.to_datetime(series.index[0]), 'D')
    current_delta = np.timedelta64(pd.to_datetime(lockdown) - pd.to_datetime(series.index[0]), 'D')
    x = np.arange(0, len(series))

In [ ]:
max_pre = np.max([
    np.timedelta64(pd.to_datetime(hubei_lockdown) - pd.to_datetime(deaths.index[0]), 'D'),
    np.timedelta64(pd.to_datetime(lombardy_lockdown) - pd.to_datetime(deaths.index[0]), 'D'),
    np.timedelta64(pd.to_datetime(spain_lockdown) - pd.to_datetime(deaths.index[0]), 'D')
])

In [ ]:
max_pre

In [ ]:
over_5_deaths = over_threshold(deaths, 5)
change = over_5_deaths.iloc[-1] - over_5_deaths.iloc[-2]

In [ ]:
data['stats']['change_deaths'] = change.to_json()

In [ ]:
data['stats']['num_countries_over_5_deaths'] = len(change)

In [ ]:
data['stats']['total_deaths'] = int(deaths.iloc[-1].sum())

In [ ]:
data['stats']['total_deaths_change'] = int(deaths.iloc[-1].sum()) - int(deaths.iloc[-2].sum())

In [ ]:
data['stats']['last_updated'] = date.today().strftime('%Y-%m-%d')

In [ ]:
data['stats']

In [ ]:
import json
with open('../../frontend/src/data/data.json', 'w') as f:
    json.dump(data, f)